In [2]:
from mysql_config import get_mysql_connection, close_mysql_connection

connection = get_mysql_connection()

if connection:
    print("Conexión a MySQL exitosa!")
    close_mysql_connection(connection)
else:
    print("No se pudo conectar a MySQL.")

Host: None, User: None, Password: None, Database: None, Port: 3306
Error al conectar a MySQL: 2017 (HY000): Can't open named pipe to host: .  pipe: MySQL (2)
No se pudo conectar a MySQL.
